# Inroduction


In [1]:
!pip install pathway bokeh --quiet # This cell may take a few seconds to execute.

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.4/149.4 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 MB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.6/77.6 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 777.6/777.6 kB 24.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.5/26.5 MB 41.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.3/135.3 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.6/244.6 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.4/318.4 kB 17.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 985.8/985.8 kB 32.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from datetime import datetime
import pathway as pw
import bokeh.plotting
import panel as pn

# Step 1: Load Dataset

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
# Load dataset with parking logs
df = pd.read_csv('/content/drive/MyDrive/SA_2025/dataset.csv')


In [11]:
# Combine date and time columns into a proper timestamp
df['Timestamp'] = pd.to_datetime(df['LastUpdatedDate'] + ' ' + df['LastUpdatedTime'], dayfirst=True)
# Sort records chronologically
df.sort_values(by='Timestamp', inplace=True)

 # Step 2: Preprocess Feature Columns

In [12]:
# Assign weights to different vehicle types
vehicle_weights = {'car': 1.0, 'bike': 0.7, 'truck': 1.5, 'cycle': 0.5}
df['VehicleWeight'] = df['VehicleType'].map(vehicle_weights)

In [13]:
# Convert traffic condition to numeric for modeling
traffic_mapping = {'low': 1, 'average': 2, 'high': 3}
df['TrafficLevel'] = df['TrafficConditionNearby'].map(traffic_mapping)

In [14]:
# Normalize occupancy to represent lot fullness
df['OccupancyRatio'] = df['Occupancy'] / df['Capacity']

# Step 3: Pricing Functions

In [15]:
# Model 1: Baseline linear increase based on occupancy
def baseline_price(prev_price, occupancy, capacity, alpha=0.1):
    return prev_price + alpha * (occupancy / capacity)

In [16]:
# Model 2: Demand-based pricing function
def demand_price(base_price, occupancy, capacity, queue, traffic, is_special, vehicle_type):
    # Feature weights for demand computation
    alpha, beta, gamma, delta, epsilon = 0.5, 0.3, 0.2, 0.1, 0.15
    demand = (
        alpha * (occupancy / capacity) +
        beta * queue -
        gamma * traffic +
        delta * is_special +
        epsilon * vehicle_weights.get(vehicle_type, 1.0)
    )
    # Normalize and cap demand
    norm_demand = min(max(demand / 10, 0), 1)
    # Calculate price based on base price
    price = base_price * (1 + 0.8 * norm_demand)
    # Ensure price remains within 0.5x to 2x r
    return min(max(price, base_price * 0.5), base_price * 2.0)

# Step 4: Apply Pricing Model to All Records

In [17]:
base_price = 10
df['Price'] = base_price  # Initial price

# Apply demand-based price model to each record
for i in range(1, len(df)):
    row = df.iloc[i]
    price = demand_price(
        base_price,
        row['Occupancy'],
        row['Capacity'],
        row['QueueLength'],
        row['TrafficLevel'],
        row['IsSpecialDay'],
        row['VehicleType']
    )
    df.at[i, 'Price'] = price


/tmp/ipython-input-17-2886326240.py:16: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '10.543188405797101' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.at[i, 'Price'] = price


In [18]:
df[['Timestamp', 'SystemCodeNumber', 'Occupancy', 'Price']].to_csv('dynamic_prices.csv', index=False)
df[['Timestamp', 'Occupancy', 'Price']].head()


,Timestamp,Occupancy,Price
0,2016-10-04 07:59:00,61,10.000000
7872,2016-10-04 07:59:00,178,11.089922
5248,2016-10-04 07:59:00,237,11.140870
3936,2016-10-04 07:59:00,264,11.670881
17056,2016-10-04 07:59:00,614,11.048986


# Step 5: Visualization

In [19]:
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import HoverTool

output_notebook()

In [20]:
# Sample plot for one parking lot
lot_id = df['SystemCodeNumber'].unique()[0]
df_lot = df[df['SystemCodeNumber'] == lot_id]

p = figure(title=f"Dynamic Price Over Time for {lot_id}",
           x_axis_type='datetime', width=800, height=350)

p.line(df_lot['Timestamp'], df_lot['Price'], line_width=2, color='green', legend_label='Price')
p.line(df_lot['Timestamp'], df_lot['Occupancy'], line_width=2, color='blue', legend_label='Occupancy', alpha=0.6)

p.xaxis.axis_label = 'Time'
p.yaxis.axis_label = 'Value'
p.add_tools(HoverTool(tooltips=[("Time", "@x{%F %T}"), ("Price", "@y")],
                      formatters={"@x": "datetime"}, mode='vline'))

p.legend.location = "top_left"
show(p)

In [21]:
from bokeh.palettes import Category10

# Plot 2: Price comparison for multiple lots
p2 = figure(title="Price Comparison Between Parking Lots",
            x_axis_type='datetime', width=800, height=350)

# Select up to 4 unique parking lots
lot_ids = df['SystemCodeNumber'].unique()[:4]
colors = Category10[10]

for idx, lot in enumerate(lot_ids):
    df_lot = df[df['SystemCodeNumber'] == lot]
    p2.line(df_lot['Timestamp'], df_lot['Price'],
            line_width=2, color=colors[idx], legend_label=lot)

p2.xaxis.axis_label = 'Time'
p2.yaxis.axis_label = 'Price ($)'
p2.legend.location = 'top_left'
show(p2)


In [ ]:
import time

# Simulated real-time price stream
print("Starting real-time simulation...")

stream_base_price = 10
for i in range(0, len(df), 10):  # simulate in batches of 10 rows
    batch = df.iloc[i:i+10].copy()
    for j, row in batch.iterrows():
        price = demand_price(
            stream_base_price,
            row['Occupancy'],
            row['Capacity'],
            row['QueueLength'],
            row['TrafficLevel'],
            row['IsSpecialDay'],
            row['VehicleType']
        )
        print(f"{row['Timestamp']} | Lot: {row['SystemCodeNumber']} | Price: ${price:.2f}")
    time.sleep(0.5)  # simulate delay


Streaming output truncated to the last 5000 lines.
2016-11-28 08:01:00 | Lot: BHMEURBRD01 | Price: $10.36
2016-11-28 08:01:00 | Lot: Others-CCCPS8 | Price: $10.27
2016-11-28 08:01:00 | Lot: Broad Street | Price: $10.31
2016-11-28 08:01:00 | Lot: BHMBCCMKT01 | Price: $10.15
2016-11-28 08:01:00 | Lot: Others-CCCPS202 | Price: $10.75
2016-11-28 08:01:00 | Lot: Others-CCCPS105a | Price: $10.52
2016-11-28 08:32:00 | Lot: BHMEURBRD01 | Price: $10.35
2016-11-28 08:32:00 | Lot: Others-CCCPS105a | Price: $10.57
2016-11-28 08:32:00 | Lot: Shopping | Price: $10.85
2016-11-28 08:32:00 | Lot: BHMMBMMBX01 | Price: $10.35
2016-11-28 08:32:00 | Lot: BHMBCCMKT01 | Price: $10.18
2016-11-28 08:32:00 | Lot: Broad Street | Price: $10.45
2016-11-28 08:32:00 | Lot: Others-CCCPS119a | Price: $10.44
2016-11-28 08:32:00 | Lot: Others-CCCPS135a | Price: $10.86
2016-11-28 08:32:00 | Lot: BHMBCCTHL01 | Price: $10.42
2016-11-28 08:32:00 | Lot: Others-CCCPS202 | Price: $10.53
2016-11-28 08:32:00 | Lot: BHMNCPHST01 |